# Plotly Pane 
This notebook tests the responsive anywidgets-based way of displaying a plotly plot. 


## 0) Imports and robust loading

This cell tries a couple of import paths.

If it fails, edit the import section to match your project layout.



In [1]:
from pathlib import Path
import sys
ROOT = Path.cwd().resolve().parents[1]  
sys.path.insert(0, str(ROOT))
from gu_toolkit.PlotlyPane import  *


## 1) Create an interactive plot that is responsive

- [ ] Check that the plot resizes correctly on window width change
- [ ]  Check that the plot resizes correctly when the width changes because of opening a side pane

In [2]:
# Assumes PlotlyPane / PlotlyPaneStyle (and _uid if you use it elsewhere) are already defined.

import ipywidgets as W
from IPython.display import display, Javascript
import numpy as np
import plotly.graph_objects as go
from gu_toolkit.PlotlyPane import  *
import uuid


def _uid(n: int = 8) -> str:
    """
    Return a short random hex identifier.

    Parameters
    ----------
    n:
        Number of hex characters to return. Default is 8.

    Notes
    -----
    This helper is not used by the public API in this file, but is commonly
    useful when you want to generate per-instance CSS classes/selectors for
    complex widget layouts.
    """
    return uuid.uuid4().hex[:n]


# ---- Plotly FigureWidget (kept as-is; driver handles resizing) ----
figw = go.FigureWidget()
figw.add_scatter(x=[], y=[], mode="lines", name="sin")

figw.update_layout(
    autosize=True,
    showlegend=False,
    margin=dict(l=8, r=8, t=36, b=10, pad=0, autoexpand=True),
    title=dict(text="Sine", x=0.5),
)
figw.update_xaxes(automargin=True)
figw.update_yaxes(automargin=True)

# ---- Plot pane: styled area + anywidget resize driver inside ----
pane = PlotlyPane(
    figw,
    style=PlotlyPaneStyle(padding_px=0, border="1px solid #ddd", border_radius_px=8),
    autorange_mode="none",   # try "once" or "always"
    defer_reveal=True,
    debounce_ms=60,
)

# ---- Container-query responsive layout (responds to OUTPUT AREA width, not viewport) ----
_UID = _uid()
scope = f"demo-{_UID}"
cname = f"cq-{_UID}"  # unique container name to avoid collisions
# Change only the CSS: make narrow/vertical plot height 30% smaller than 30vh -> 21vh.
# (i.e., 30vh * 0.7 = 21vh)

NARROW_PLOT_VH = 40  # 30vh reduced by ~30%

css = W.HTML(f"""
<style>
.{scope}.wrap {{
  container-type: inline-size;
  container-name: {cname};
  width: 100%;
}}

.{scope} .root {{
  display:flex;
  flex-direction:row;
  gap:12px;
  width:100%;
  height:70vh;
  box-sizing:border-box;
  min-height:0;
}}

.{scope} .plotcell {{
  flex:1 1 65%;
  min-width:0;
  min-height:0;
  display:flex;
  flex-direction:column;
  overflow:hidden;
}}

.{scope} .side {{
  flex:0 1 360px;
  min-width:280px;
  max-width:40%;
  display:flex;
  flex-direction:column;
  gap:12px;
  min-height:0;
}}

.{scope} .panel {{
  border:1px solid #ddd;
  border-radius:8px;
  padding:8px;
  overflow:auto;
  min-height:0;
  box-sizing:border-box;
}}
.{scope} .controls {{ flex:2 1 0; }}
.{scope} .sideouts {{ flex:1 1 0; }}

@container {cname} (max-width: 900px) {{
  .{scope} .root {{ flex-direction:column; height:auto; }}
  .{scope} .plotcell {{
    height:{NARROW_PLOT_VH}vh;
    flex:0 0 auto;      /* don't let flex override the fixed height */
  }}
  .{scope} .side {{ max-width:100%; min-width:0; }}
  .{scope} .panel {{ max-height:35vh; }}
}}

.{scope}.is-narrow .root {{ flex-direction:column; height:auto; }}
.{scope}.is-narrow .plotcell {{
  height:{NARROW_PLOT_VH}vh;
  flex:0 0 auto;
}}
.{scope}.is-narrow .side {{ max-width:100%; min-width:0; }}
.{scope}.is-narrow .panel {{ max-height:35vh; }}
</style>
""")



plot_cell = W.Box(
    [pane.widget],
    layout=W.Layout(
        width="100%",
        #height="100%",
        min_width="0",
        min_height="0",
        display="flex",
        flex_flow="column",
        overflow="hidden",
    ),
)
plot_cell.add_class("plotcell")

controls_box = W.VBox([], layout=W.Layout(width="100%", min_height="0"))
controls_panel = W.Box([controls_box], layout=W.Layout(width="100%", min_height="0"))
controls_panel.add_class("panel")
controls_panel.add_class("controls")

side_out = W.Output()
sideouts_panel = W.Box([side_out], layout=W.Layout(width="100%", min_height="0"))
sideouts_panel.add_class("panel")
sideouts_panel.add_class("sideouts")

side_col = W.Box(
    [controls_panel, sideouts_panel],
    layout=W.Layout(width="100%", min_height="0"),
)
side_col.add_class("side")

root = W.Box(
    [plot_cell, side_col],
    layout=W.Layout(width="100%", min_height="0"),
)
root.add_class("root")

ui = W.Box([root], layout=W.Layout(width="100%"))
ui.add_class(scope)
ui.add_class("wrap")  # IMPORTANT: makes ui the container for @container rules

# ---- Controls + wiring ----
freq = W.FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description="freq", continuous_update=False)
amp  = W.FloatSlider(value=1.0, min=0.0, max=3.0, step=0.05, description="amp", continuous_update=False)
npts = W.IntSlider(value=400, min=50, max=5000, step=50, description="n", continuous_update=False)
btn_redraw = W.Button(description="Redraw")
btn_reflow = W.Button(description="Reflow plot")

controls_box.children = [
    W.HTML("<b>Controls</b>"),
    freq, amp, npts,
    W.HBox([btn_redraw, btn_reflow], layout=W.Layout(gap="8px")),
]

def redraw(*_):
    x = np.linspace(0, 2*np.pi, int(npts.value))
    y = amp.value * np.sin(freq.value * x)
    with figw.batch_update():
        figw.data[0].x = x
        figw.data[0].y = y
        figw.layout.title = f"Sine (freq={freq.value}, amp={amp.value})"
    with side_out:
        side_out.clear_output(wait=True)
        print(f"freq={freq.value}, amp={amp.value}, n={npts.value}")

for w in (freq, amp, npts):
    w.observe(redraw, "value")
btn_redraw.on_click(redraw)
btn_reflow.on_click(lambda *_: pane.reflow())

redraw()

display(css, ui)

# Optional: post-display reflow (your reflow logic remains unchanged)
pane.reflow()

# Optional: fallback for browsers/environments without container queries
# (does NOT affect Plotly; only toggles .is-narrow for layout)
display(Javascript(f"""
(() => {{
  const root = document.querySelector('.{scope}');
  if (!root) return;

  // If container queries are supported, do nothing (CSS handles it).
  if (window.CSS && CSS.supports && CSS.supports('container-type: inline-size')) return;

  const THRESH = 900;
  let last = -1;

  function tick() {{
    const w = root.getBoundingClientRect().width | 0;
    if (w !== last) {{
      last = w;
      root.classList.toggle('is-narrow', w <= THRESH);
    }}
    requestAnimationFrame(tick);
  }}
  requestAnimationFrame(tick);
}})();
"""))


HTML(value="\n<style>\n.demo-c1d804a8.wrap {\n  container-type: inline-size;\n  container-name: cq-c1d804a8;\n…

Box(children=(Box(children=(Box(children=(Box(children=(Box(children=(FigureWidget({
    'data': [{'mode': 'li…

<IPython.core.display.Javascript object>